<table style="width:100%; background-color:#FFFFFF" align="left">
 <tr>
   <td style="text-align:left; background-color:#FFFFFF">
       <p>Prof. Dr. Stefan Decker <br>
        M. Slupczynski <br>
        L. Neumann <br>
        M. Kißgen <br>
        T. Holzheim</p>
        <p><strong>09-05-2025</strong></p>
     </td>
   <td style="text-align:right; background-color:#FFFFFF">
     <div>
       <img src="https://git.rwth-aachen.de/i5/teaching/dbis-raw/-/raw/main/dbis-logo.png" width="500"/>
     </div>
    </td>
 </tr>
</table>
<div class="clearfix"></div>
<center>
    <h1>Datenbanken und Informationssysteme (SoSe 2025) </h1>
    <h1>Übung 4 - Tupel- und Domänenkalkül</h1>
</center>

<center>
    <h4>Abgabe bis 16-05-2025 14:15 Uhr</h4>
    <strong>Zu spät eingereichte Übungen werden nicht berücksichtigt.</strong>
</center>

<div class="hidden-print"><hr />
<h3> Wichtige Hinweise</h3>
<ul>
<li>Bei Nichtbeachtung dieser Hinweise wird die Abgabe mit 0 Punkten bewertet!</li>
<li>Bitte laden Sie Ihre digitalen Lösungen mit diesem Notebook ins Moodle hoch.</li>
<li>Bitte geben Sie Namen, Matrikelnummern auf der schriftlichen Lösung an.</li>
</ul></div>

<strong>Die Lösung zu diesem Übungsblatt wird in den Übungen am 16. und 19. Mai vorgestellt. <br /> 
Bitte beachten Sie auch die aktuellen Ankündigungen im Moodle-Lernraum zur Vorlesung. </strong> <br />
<em>* bezeichnet Bonusaufgaben.</em>
<hr />

<h3>Dokumentations-Notebook</h3>
Wir empfehlen ihnen das zu dieser Übung zugehörige Dokumentations-Notebook zu lesen. Zusätzlich zur der Handhabung der verschiedenen verwendeten Tools werden dort zentrale Konzepte der Vorlesung noch einmal wiederholt. Sie finden das Notebook im <a href="https://moodle.rwth-aachen.de/course/view.php?id=48593">Moodle Lernraum</a>.

<h3>Abgabegruppe</h3>
Es ist möglich die Abgabe in Gruppen von bis zu fünf Personen zu bearbeiten. <strong>Jeder Gruppenteilnehmer muss für sich ein Notebook hochladen</strong>. Wenn Sie in einer Abgabegruppe arbeiten, dürfen die Notebooks der Gruppenteilnehmer den gleichen Inhalt aufweisen. Wenn Sie in einer Gruppe arbeiten, tragen Sie hier bitte alle Gruppenteilnehmer inklusive sich selbst ein. <strong>Eine inkonsistente Angabe von Gruppenmitgliedern bei den Teilnehmern führt im Zweifel zu 0 Punkten für alle Gruppenteilnehmer!</strong> Es ist möglich zwischen den Abgaben die Gruppen zu wechseln.

<h4>Andere Gruppenteilnehmer (Wenn Einzelabgabe, bitte leer lassen)</h4>
<ul>
    <li>Neo Ahrens, 456647</li>
    <li>Yorck Heilmann, 456599</li>
    <li>Christian Bick, 456513</li>
    <li>Michael Hübner, 457202</li>
    <li>Kai Seeling, 456990</li>
</ul>

<div class="alert alert-info hidden-print">Die folgende Codezelle sollte zu Beginn der Bearbeitung der Übung ausgeführt werden. <br /> Am Ende sollte <b>"OK, es kann los gehen!"</b> stehen. Dies kann ein paar Minuten dauern.</div>
<div class="alert alert-danger">
    Sie sollten die weiteren Zellen erst ausführen, wenn <b>"OK, es kann los gehen!"</b> am Ende der Ausgabe der nächsten Zelle steht. <br /> Andere Ausgaben dieser Zelle können ignoriert werden. <br />
</div>

In [2]:
%pip install -q "dbis-exc-manager~=1.0.2" "dbis-relational-calculus>=1.0.12"
from IPython.display import display, HTML
import warnings
try:
    # import libraries
    import os
    from excmanager.Task import Exercise, Task, SubTask
    # initialize task structure
    exc = Exercise(4)
    task1 = Task(exc, "4.1")
    task2 = Task(exc, "4.2")
    task3 = Task(exc, "4.3")
    tasks_ready = {"4.1": {}, "4.2": {}, "4.3": {}}

    # load hotel.sql in sqlite3 with :memory: --> for RC tasks
    if 'sql_con' in globals():
        sql_con.close()
    with open('assets/Chinook_Sqlite.sql', 'r', encoding="utf8") as f:
        schema = f.read()
    import sqlite3
    if os.path.exists('assets/chinook.db'): os.remove('assets/chinook.db')
    sql_con = sqlite3.connect('assets/chinook.db')
    cursor = sql_con.cursor()
    cursor.executescript(schema)
except Exception as e:
    display(HTML(
        f"<div class='alert alert-danger'><h2>Etwas ist beim import schief gelaufen:</h1>{e}</div>"))
    raise e
else:
    display(
        HTML("<div class='alert alert-success'><h1>OK, es kann los gehen!</h1></div>"))

Note: you may need to restart the kernel to use updated packages.


<div class="pull-left">

<h2> Aufgabe 4.1 (Tupelkalkül) </h2>

</div>

<div class="pull-right">

<h3> (8 Punkte) </h3>

</div>

Gegeben sei das folgende abgespeckte relationale Datenbankschema der Chinook-Datenbank:

**Relationen:**
 - $Album(\underline{AlbumId}, Title, ArtistId)$
 - $Artist(\underline{ArtistId}, Name)$
 - $Customer(\underline{CustomerId}, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId)$
 - $Employee(\underline{EmployeeId}, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email)$
 - $Genre(\underline{GenreId}, Name)$
 - $Invoice(\underline{InvoiceId}, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total)$
 - $InvoiceLine(\underline{InvoiceLineId}, InvoiceId, TrackId, UnitPrice, Quantity)$
 - $Playlist(\underline{PlaylistId}, Name)$
 - $PlaylistTrack(\underline{PlaylistId}, TrackId)$
 - $Track(\underline{TrackId}, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)$

Interrelationale Abhängigkeiten:
 - $Album[ArtistId] \subseteq Artist[ArtistId]$
 - $Customer[SupportRepId] \subseteq Employee[EmployeeId]$
 - $Invoice[CustomerId] \subseteq Customer[CustomerId]$
 - $InvoiceLine[InvoiceId] \subseteq Invoice[InvoiceId]$
 - $InvoiceLine[TrackId] \subseteq Track[TrackId]$
 - $PlaylistTrack[PlaylistId] \subseteq Playlist[PlaylistId]$
 - $PlaylistTrack[TrackId] \subseteq Track[TrackId]$
 - $Track[GenreId] \subseteq Genre[GenreId]$
 - $Track[AlbumId] \subseteq Album[AlbumId]$
 - $Employee[ReportsTo] \subseteq Employee[EmployeeId]$

Zusätzlich gelten:
 - Alle Attribute die eine Zeitangabe erhalten sind vom Typ `int` und repräsentieren einen Unix-Timestamp mit Sekundenauflösung.

---

Verwenden Sie die Library für das Relationale Kalkül um die folgenden Anfragen im Tupelkalkül zu formulieren, Sie finden das Doku-Relationales-Kalkül.ipnyb Notebook im <a href="https://moodle.rwth-aachen.de/course/view.php?id=48593">Moodle Lernraum</a>.<br>
Die Ausgabe beim Tupelkalkül ist in Großbuchstaben, da ganze Tupel aus der Relation ausgegeben werden. Beim Domänenkalkül erfolgt die Ausgabe in Kleinbuchstaben, da nur einzelne Attribute der Relation zurückgegeben werden.

<div class="alert alert-block alert-danger">
<b>Hinweis</b>: Die Spaltennamen in der resultierenden Tabelle werden durch die jeweiligen Attribute bestimmt. In den Tupelkalkül sind diese durch die obigen Relationen bereits vorgegeben. Desweiteren ist die korrekte Reihenfolge der Spaltennamen zu beachten.
</div>

<div class="pull-left">
<h3>Aufgabe 4.1 a)</h3>
</div>

<div class="pull-right">
<h3>(1 Punkt)</h3>
</div>

Listen Sie alle Albumtitel und IDs auf, deren Artist `Eric Clapton` ist.<br>
Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `Title`, `AlbumId`.

In [5]:
from relational_calculus.tuple_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable.
expression = None

# IHR CODE HIER
a = Variable('a', 'Album')
t = Variable('t', 'Artist')

formula = And(a,Exists(t, And(Equals((t,"Name"), "Eric Clapton"), Equals((a,"ArtistId"), (t, "ArtistId")))))
result = Result([a['Title'], a['AlbumId']])
expression = TupleCalculus(result, formula)


# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte

subtask = SubTask(task1, 'a', points=1)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["Title", "AlbumId"]

for attr in list(attributes):
    for x in expression.result.attributes:
        if not isinstance(x,Variable):
            if attr==x[1]:
                attributes.remove(x[1])

tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")    
        
# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))



<IPython.core.display.Math object>

| Title                |   AlbumId |
|----------------------|-----------|
| The Cream Of Clapton |        72 |
| Unplugged            |        73 |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.1 b)</h3>
</div>

<div class="pull-right">
<h3>(2 Punkte)</h3>
</div>

Listen Sie alle Tracks mit TrackIds und die Namen ihrer Genres auf, die zum Album `Black Sabbath` gehören.<br>
Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `TrackId`, `Name`. 

In [6]:
from relational_calculus.tuple_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable.
expression = None

# IHR CODE HIER
a = Variable('a', 'Album')
t = Variable('t', 'Track')
g = Variable('g', 'Genre')

sameGenre = Exists(g, Equals((g, "GenreId"), (t, "GenreID")))
sameAlbum = Exists(a , And(Equals((a,"AlbumId"), (t,"AlbumId")), Equals((a,"Title"), "Black Sabbath")))
formula = And(t, And(sameGenre, sameAlbum))
result = Result([t['TrackId'], g['Name']])
expression = TupleCalculus(result, formula)





# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task1, 'b', points=2)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["TrackId", "Name"]

for attr in list(attributes):
    for x in expression.result.attributes:
        if not isinstance(x,Variable):
            if attr==x[1]:
                attributes.remove(x[1])

tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 2, "Die Anzahl der zurückgegebenen Attribute ist falsch."

<IPython.core.display.Math object>

|   TrackId | Name   |
|-----------|--------|
|       149 | Metal  |
|       150 | Metal  |
|       151 | Metal  |
|       152 | Metal  |
|       153 | Metal  |
|       154 | Metal  |
|       155 | Metal  |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.1 c)</h3>
</div>

<div class="pull-right">
<h3>(2 Punkte)</h3>
</div>

Bestimmen Sie den kürzesten Track pro mediaType.<br>

Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `MediaTypeId`, `TrackId`, `Name`, `Milliseconds`.

In [7]:
from relational_calculus.tuple_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable.
expression = None

# IHR CODE HIER
t = Variable('t', 'Track')
x = Variable('x', 'Track')
s = Variable('s', 'Track')

#allShorter = Forall(x, And(Equals((x,"MediaTypeId"), (t,"MediaTypeId")), LessEquals((t,"Milliseconds"), (x,"Milliseconds"))))
allShorter = Forall(x, Or(Not(Equals((x,"MediaTypeId"), (t,"MediaTypeId"))), LessEquals((t,"Milliseconds"), (x,"Milliseconds"))))
#allShorter = Equals((t,"MediaTypeId"), 3)
formula = And(t, allShorter)

result = Result([t['MediaTypeId'], t['TrackId'], t["Name"], t["Milliseconds"]])
expression = TupleCalculus(result, formula)






# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task1, 'c', points=2)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["MediaTypeId", "TrackId", "Name", "Milliseconds"]


for attr in list(attributes):
    for x in expression.result.attributes:
        if not isinstance(x,Variable):
            if attr==x[1]:
                attributes.remove(x[1])

tasks_ready[subtask.task.task][subtask.subtask] = True
if not (len(attributes)==0):
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht aus beiden Relationen ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 4, "Die Anzahl der zurückgegebenen Attribute ist falsch."

<IPython.core.display.Math object>

|   MediaTypeId |   TrackId | Name                                            |   Milliseconds |
|---------------|-----------|-------------------------------------------------|----------------|
|             1 |      2461 | É Uma Partida De Futebol                        |           1071 |
|             3 |      3339 | LOST Season 4 Trailer                           |         112712 |
|             5 |      3356 | Muita Bobeira                                   |         172710 |
|             4 |      3496 | Étude 1, In C Major - Preludio (Presto) - Liszt |          51780 |
|             2 |      3501 | L'orfeo, Act 3, Sinfonia (Orchestra)            |          66639 |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.1 d)</h3>
</div>

<div class="pull-right">
<h3>(3 Punkte)</h3>
</div>

Geben Sie an, welche Tracks aus dem Album `Slaves And Masters` mindestens einmal in die `USA` verkauft wurden.

Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `Title`, `Name`.

In [8]:
from relational_calculus.tuple_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable.
expression = None

# IHR CODE HIER
t = Variable('t', 'Track')
a = Variable('a', 'Album')
c = Variable('c', 'Customer')
i = Variable('i', 'Invoice')
l = Variable('l', 'InvoiceLine')

customer = Exists(l, Exists(i, And(And(Equals((l, "InvoiceId"), (i, "InvoiceId")), Equals((i,"BillingCountry"), "USA")), Equals((l, "TrackId"), (t,"TrackId")))))
album = Exists(a, And(Equals((a,"Title"), "Slaves And Masters"), Equals((t, "AlbumId"), (a, "AlbumId"))))
formula = And(t, And(customer, album))

result = Result([a['Title'], t['Name']])
expression = TupleCalculus(result, formula)





# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task1, 'd', points=3)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
print()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["Title", "Name"]

for attr in list(attributes):
    for x in expression.result.attributes:
        if not isinstance(x,Variable):
            if attr.lower()==x[1].lower():
                attributes.remove(x[1])

tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 2, "Die Anzahl der zurückgegebenen Attribute ist falsch."

<IPython.core.display.Math object>

| Title              | Name           |
|--------------------|----------------|
| Slaves And Masters | Fortuneteller  |
| Slaves And Masters | King Of Dreams |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">

<h2> Aufgabe 4.2 (Domänenkalkül) </h2>

</div>

<div class="pull-right">

<h3> (6 Punkte) </h3>

</div>

Gegeben sei das folgende abgespeckte relationale Datenbankschema der Chinook-Datenbank:

**Relationen:**
 - $Album(\underline{AlbumId}, Title, ArtistId)$
 - $Artist(\underline{ArtistId}, Name)$
 - $Customer(\underline{CustomerId}, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId)$
 - $Employee(\underline{EmployeeId}, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email)$
 - $Genre(\underline{GenreId}, Name)$
 - $Invoice(\underline{InvoiceId}, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total)$
 - $InvoiceLine(\underline{InvoiceLineId}, InvoiceId, TrackId, UnitPrice, Quantity)$
 - $Playlist(\underline{PlaylistId}, Name)$
 - $PlaylistTrack(\underline{PlaylistId}, TrackId)$
 - $Track(\underline{TrackId}, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)$

Interrelationale Abhängigkeiten:
 - $Album[ArtistId] \subseteq Artist[ArtistId]$
 - $Customer[SupportRepId] \subseteq Employee[EmployeeId]$
 - $Invoice[CustomerId] \subseteq Customer[CustomerId]$
 - $InvoiceLine[InvoiceId] \subseteq Invoice[InvoiceId]$
 - $InvoiceLine[TrackId] \subseteq Track[TrackId]$
 - $PlaylistTrack[PlaylistId] \subseteq Playlist[PlaylistId]$
 - $PlaylistTrack[TrackId] \subseteq Track[TrackId]$
 - $Track[GenreId] \subseteq Genre[GenreId]$
 - $Track[AlbumId] \subseteq Album[AlbumId]$
 - $Employee[ReportsTo] \subseteq Employee[EmployeeId]$

Zusätzlich gelten:
 - Alle Attribute die eine Zeitangabe erhalten sind vom Typ `int` und repräsentieren einen Unix-Timestamp mit Sekundenauflösung.
 - Wie im Doku-Notebook beschrieben, besteht die Möglichkeit nicht genutzte Variablen in einer Relation einfach mit `_` zu kennzeichnen. In diesem Falle müssen die Variablen an deren Stelle einen ein _ steht, auch mit keinem Existenz-/Allquantor instanziiert werden. **Dies ist nur eine Funktion der Library und in der Vorlesung nicht definiert.**

---

Verwenden Sie die Library für das Relationale Kalkül um die folgenden Anfragen im Domänenkalkül zu formulieren, Sie finden das Doku-Relationales-Kalkül.ipnyb Notebook im <a href="https://moodle.rwth-aachen.de/course/view.php?id=48593">Moodle Lernraum</a>.<br>
Die Ausgabe beim Tupelkalkül erscheint in Großbuchstaben, da ganze Tupel aus der Relation ausgegeben werden. Beim Domänenkalkül erfolgt die Ausgabe in Kleinbuchstaben, da nur einzelne Attribute der Relation zurückgegeben werden.

<div class="alert alert-block alert-danger">
<b>Hinweis</b>: Die Spaltennamen in der resultierenden Tabelle werden durch den Namen der Domänenvariablen bestimmt. Es ist darauf zu achten, dass diese Namen so gewählt werden, wie es die jeweilige Aufgabenstellung vorgibt! Desweiteren ist die korrekte Reihenfolge der Spaltennamen zu beachten.
</div>

<div class="pull-left">
<h3>Aufgabe 4.2 a)</h3>
</div>

<div class="pull-right">
<h3>(1 Punkte)</h3>
</div>

Geben Sie den Employee aus, der die Kundin `Kara Nielsen` betreut. <br>

Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `employeeid`, `lastname`, `firstname`. <br>

Hinweis: Ein entsprechende Variable `employeeid` ist Ihnen bereits gegeben.

In [10]:
from relational_calculus.domain_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable
expression = None
employeeid = Variable('employeeid')

# IHR CODE HIER
lastname = Variable('lastname')
firstname = Variable('firstname')
supportrepid = Variable('supportrepid')


formula = Exists({employeeid, lastname, firstname}, And(Tuple("Employee", [employeeid, lastname, firstname,  None, None, None, None, None, None, None, None, None, None, None, None]), Exists(supportrepid, And(Tuple("Customer", [None, "Kara", "Nielsen", None, None, None, None, None, None, None, None, None, supportrepid]), Equals(employeeid, supportrepid)))))

result = Result([employeeid, lastname, firstname])

expression = DomainCalculus(result, formula)

# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task2, 'a', points=1)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["employeeid", "lastname", "firstname"]

for attr in list(attributes):
    for x in expression.result.variables:
        if attr.lower()==x.name.lower():
            attributes.remove(x.name)
                
tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 3, "Die Anzahl der zurückgegebenen Attribute ist falsch."

<IPython.core.display.Math object>

|   employeeid | lastname   | firstname   |
|--------------|------------|-------------|
|            4 | Park       | Margaret    |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.2 b)</h3>
</div>

<div class="pull-right">
<h3>(2 Punkte)</h3>
</div>

Listen Sie alle Customers (mit Id, Vor- und Nachname) auf, die mindestens 2 Einkäufe (invoice) mit Beiträgen (jeweils) > `9` getätigt haben.<br>
Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `customerid`, `firstname`, `lastname`.

Hinweis: Ein entsprechende Variable `customerid` ist Ihnen bereits gegeben.

In [8]:
from relational_calculus.domain_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable
expression = None
customerid = Variable('customerid')

# IHR CODE HIER
firstname = Variable("firstname")
lastname = Variable("lastname")
invoice1 = Variable("invoice1")
invoiceCustomer1 = Variable("invoiceCustomer1")
invoice2 = Variable("invoice2")
invoiceCustomer2 = Variable("invoiceCustomer2")
total1 = Variable("total1")
total2 = Variable("total2")

customerTuple = Tuple("Customer", [customerid, firstname, lastname, None, None, None, None, None, None, None, None, None, None])
invoice1Tuple = Tuple("Invoice", [invoice1, customerid, None, None, None, None, None, None, total1])
invoice2Tuple = Tuple("Invoice", [invoice2, customerid, None, None, None, None, None, None, total2])


#formula = Exists({invoice1, customerid} , And(invoice1Tuple, Exists({firstname, lastname}, And(customerTuple, And(Not(Equals(invoice1, invoice2)), And(GreaterThan(total1, 9), GreaterThan(total2, 9)))))))















customerTuple = Tuple("Customer", [customerid, firstname, lastname, None, None, None, None, None, None, None, None, None, None])
invoice1Tuple = Tuple("Invoice", [invoice1, customerid, None, None, None, None, None, None, total1])
invoice2Tuple = Tuple("Invoice", [invoice2, customerid, None, None, None, None, None, None, total2])

Ex1 = Exists({invoice1}, And(invoice1Tuple, GreaterThan(total1, 9)))
Ex2 = Exists({invoice2}, And(invoice2Tuple, GreaterThan(total2, 9)))

#formula = Exists({customerid, firstname, lastname}, And(customerTuple, And(Ex1, And(Ex2, Not(Equals(invoice1, invoice2))))))






formula = Exists(
    customerid,
    And(
        customerTuple,
        Exists(invoice1,
            And(
                invoice1Tuple,
                Exists(invoice2,
                    And(
                        invoice2Tuple,
                        And(
                            Not(Equals(invoice1, invoice2)),
                            And(
                                GreaterThan(total1,9),
                                GreaterThan(total2,9)
                            )
                        )
                    )
                )
            )
        )
    )
)
                

#formula = Exists({customerid, firstname, lastname}, customerTuple)
#result = Result([customerid])
             
                                                        
                                                        
result = Result([customerid, firstname, lastname])

expression = DomainCalculus(result, formula)


# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task2, 'b', points=2)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["customerid", "firstname", "lastname"]

for attr in list(attributes):
    for x in expression.result.variables:
        if attr.lower()==x.name.lower():
            attributes.remove(x.name)
                
tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 3, "Die Anzahl der zurückgegebenen Attribute ist falsch."

<IPython.core.display.Math object>

|   customerid | firstname   | lastname   |
|--------------|-------------|------------|
|           15 | Jennifer    | Peterson   |
|           17 | Jack        | Smith      |
|           28 | Julia       | Barnett    |
|           34 | João        | Fernandes  |
|           37 | Fynn        | Zimmermann |
|           57 | Luis        | Rojas      |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.2 c)</h3>
</div>

<div class="pull-right">
<h3>(3 Punkte)</h3>
</div>

Bob hat Alice 2 Artists `Billy Cobham` und `Bruce Dickinson` empfohlen, Alice hört sehr gerne `Jazz` Musik, welche Tracks von den 2 Artists könnten Alice möglicherweise gefallen?

Die Ausgabe soll die folgenden Attribute in dieser Reihenfolge enthalten: `artistname`, `albumtitle`, `trackname`.

Hinweis: Entsprechende Variablen `albumtitle` und `trackname` sind Ihnen gegeben, suchen Sie nach den Jazz-songs von `Billy Cobham` und `Bruce Dickinson`.

In [9]:
from relational_calculus.domain_calculus import *
from IPython.display import display, Math, Markdown

#Initialisierung Variable.
expression = None
atitle = Variable('albumtitle')
tname = Variable('trackname')
artistid = Variable('artistid')
albumid = Variable('albumid')
genreid = Variable('genreid')
artistname = Variable('artistname')


# IHR CODE HIER
artist = Tuple("Artist", [artistid, artistname])
jazz = Tuple("Genre", [genreid, "Jazz"])
album = Tuple("Album", [albumid, atitle, artistid])
track = Tuple("Track", [None, tname, albumid, None, genreid, None, None, None, None])

#formula = Exists(tname, And(track, Exists(genreid, And(jazz, Or(Exists(artistid, And(billy, Exists(albumid, album))), Exists(artistid, And(bruce, Exists(albumid, album))))))))         



chooseArtist = Exists({artistname, artistid}, 
                      And(
                          artist, 
                          Or(
                              Equals(artistname, "Bruce Dickinson"), 
                              Equals(artistname, "Billy Cobham"))))


formula = Exists(genreid, And(
    jazz,
    Or(
        Exists({artistname, artistid}, And(
            artist,
            Exists(atitle, And(
                album,
                And(
                    Equals(artistname, "Billy Cobham"),
                    Exists(tname, track)
                )
            ))
        )),
        Exists({artistname, artistid}, And(
            artist,
            Exists(atitle, And(
                album,
                And(
                    Equals(artistname, "Bruce Dickinson"),
                    Exists(tname, track)
                )
            ))
        ))
    )
))



lel = Exists({tname, albumid}, And(
    track,
    Exists(genreid, And(
        jazz,
        Exists({albumid, atitle, artistname}, And(
            album,
            Or(
                Exists(artistname, And(
                    artist,
                    Equals(artistname, "Bruce Dickinson")
                )),
                Exists(artistname, And(
                    artist,
                    Equals(artistname, "Billy Cobham")
                ))
            )
        ))
    ))
))



#formula = Exists(genreid, Tuple("Genre", [genreid, "Jazz"]))
#result = Result([artistname])
                      
        
result = Result([artistname, atitle, tname])

expression = DomainCalculus(result, formula)


'''
Exists({artistname, artistid}, And(
            artist,
            Equals(artistname, "Bruce Dickinson"), And(
                Exists({albumid, atitle}, And(
                    album,
                    Exists(tname, track)
                ))
            ))
        ),
'''






# DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 Punkte
subtask = SubTask(task2, 'c', points=3)

# display expression in math mode
display(Math(str(expression)))

sql_expression = expression.to_sql()
subtask.setSolution(expression)
subtask.setData(expression)

# Prüfe Attribute
attributes = ["artistname","albumtitle", "trackname"]

for attr in list(attributes):
    for x in expression.result.variables:
        if attr.lower()==x.name.lower():
            attributes.remove(x.name)
                
tasks_ready[subtask.task.task][subtask.subtask] = True
if not len(attributes)==0:
    tasks_ready[subtask.task.task][subtask.subtask] = False
    warnings.warn(f"Geforderte(s) Attribut(e) {attributes} werden nicht ausgegeben")

# display result table
cursor = sql_con.execute(sql_expression)
attributes = [description[0] for description in cursor.description]
tuples = [list(row) for row in cursor.fetchall()]
from tabulate import tabulate
table = tabulate(tuples, attributes, tablefmt="github")
display(Markdown(table))

assert len(attributes) == 3

<IPython.core.display.Math object>

| artistname   | albumtitle               | trackname                                    |
|--------------|--------------------------|----------------------------------------------|
| Billy Cobham | The Best Of Billy Cobham | Quadrant                                     |
| Billy Cobham | The Best Of Billy Cobham | Snoopy's search-Red baron                    |
| Billy Cobham | The Best Of Billy Cobham | Spanish moss-"A sound portrait"-Spanish moss |
| Billy Cobham | The Best Of Billy Cobham | Moon germs                                   |
| Billy Cobham | The Best Of Billy Cobham | Stratus                                      |
| Billy Cobham | The Best Of Billy Cobham | The pleasant pheasant                        |
| Billy Cobham | The Best Of Billy Cobham | Solo-Panhandler                              |
| Billy Cobham | The Best Of Billy Cobham | Do what cha wanna                            |

In [ ]:
# Autograder, bitte ignorieren

<div class="pull-left">

<h2> Aufgabe 4.3 (Tupel- vs. Domänenkalkül) </h2>

</div>

<div class="pull-right">

<h3> (6 Punkte) </h3>

</div>

In dieser Aufgabe schauen wir uns die Beziehung zwischen dem Tupelkalkül und dem Domänenkalkül genauer an. Als Basis nehmen wir weiterhin das Schema von Chinook:

**Relationen:**
 - $Album(\underline{AlbumId}, Title, ArtistId)$
 - $Artist(\underline{ArtistId}, Name)$
 - $Customer(\underline{CustomerId}, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId)$
 - $Employee(\underline{EmployeeId}, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email)$
 - $Genre(\underline{GenreId}, Name)$
 - $Invoice(\underline{InvoiceId}, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total)$
 - $InvoiceLine(\underline{InvoiceLineId}, InvoiceId, TrackId, UnitPrice, Quantity)$
 - $Playlist(\underline{PlaylistId}, Name)$
 - $PlaylistTrack(\underline{PlaylistId}, TrackId)$
 - $Track(\underline{TrackId}, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice)$

Interrelationale Abhängigkeiten:
 - $Album[ArtistId] \subseteq Artist[ArtistId]$
 - $Customer[SupportRepId] \subseteq Employee[EmployeeId]$
 - $Invoice[CustomerId] \subseteq Customer[CustomerId]$
 - $InvoiceLine[InvoiceId] \subseteq Invoice[InvoiceId]$
 - $InvoiceLine[TrackId] \subseteq Track[TrackId]$
 - $PlaylistTrack[PlaylistId] \subseteq Playlist[PlaylistId]$
 - $PlaylistTrack[TrackId] \subseteq Track[TrackId]$
 - $Track[GenreId] \subseteq Genre[GenreId]$
 - $Track[AlbumId] \subseteq Album[AlbumId]$
 - $Employee[ReportsTo] \subseteq Employee[EmployeeId]$

Zusätzlich gelten:
 - Wie im Doku-Notebook beschrieben, besteht die Möglichkeit nicht genutzte Variablen in einer Relation einfach mit `_` zu kennzeichnen. In diesem Falle müssen die Variablen an deren Stelle einen ein _ steht, auch mit keinem Existenz-/Allquantor instanziiert werden. **Dies ist nur eine Funktion der Library und in der Vorlesung nicht definiert.**

<div class="pull-left">
<h3>Aufgabe 4.3 a)</h3>
</div>

<div class="pull-right">
<h3>(3 Punkte)</h3>
</div>

Gegeben ist eine Formel im Tupelkalkül: <br>

$\{[\textit{c.FirstName}, \textit{c.LastName}] \vert \textit{c} \in \textit{Customer} $ <br>
$\land \exists e(\textit{e} \in \textit{EMPLOYEE} $ <br>
$\land (\textit{c.SupportRepId} = \textit{e.EmployeeId} $<br>
$\land ( \textit{e.ReportsTo} = \textit{e.EmployeeId})))\}$

Welche der folgenden Formeln im Domänenkalkül entspricht/entsprechen der obigen Formel?<br>

__a:__ <br>
$\{\textit{FirstName}, \textit{LastName} | \exists {sid} ((\textit{Customer}(\_, \textit{FirstName}, \textit{LastName}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \textit{sid})$ <br>
$\land \exists {eid} (\textit{Employee}(\textit{eid}, \_, \_, \_, \textit{sid}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_))))\}$ <br>


__b:__ <br>
$\{\textit{FirstName}, \textit{LastName} | \exists {cid} ((\textit{Customer}(\textit{cid}, \textit{FirstName}, \textit{LastName}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \textit{cid})$ <br>
$\land \exists {eid} (\textit{Employee}(\textit{eid}, \_, \_, \_, \textit{eid}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_))))\}$ <br>

__c:__ <br>
$\{\textit{FirstName}, \textit{LastName} | \exists {sid} ((\textit{Customer}(\_, \textit{FirstName}, \textit{LastName}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \textit{sid})$ <br>
$\land \textit{Employee}(\textit{sid}, \_, \_, \_, \textit{sid}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_)))\}$ <br>

__d:__ <br>
$\{\textit{FirstName}, \textit{LastName} | \exists {sid} ((\textit{Customer}(\_, \textit{FirstName}, \textit{LastName}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \textit{sid})$ <br>
$\land \textit{Employee}(\textit{sid}, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_, \_)))\}$ <br>

In [13]:
 # result ist hier eine String-Menge, z.B. result = {'a','b'}. Füllen sie die Menge unter dem Kommentar mit Ihrer Lösung.
result = set()

# IHR CODE HIER
result = {"c"}
#result = {}

### DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 PUNKTE!
subtask = SubTask( task3, "a", points = 3 )

tasks_ready[subtask.task.task][subtask.subtask] = True
for element in result:
    if not isinstance(element, str):
        tasks_ready[subtask.task.task][subtask.subtask] = False
        warnings.warn(f"Die Menge darf nur aus Strings bestehen.")
        break
    if not element in ["a","b","c","d"]:
        tasks_ready[subtask.task.task][subtask.subtask] = False
        warnings.warn(f"Die Menge darf nur die Antwortmöglichkeiten a,b,c,d enthalten.")
        break

subtask.setSolution(result)
subtask.setData(result)

In [ ]:
 # Autograder, bitte ignorieren

<div class="pull-left">
<h3>Aufgabe 4.3 b)</h3>
</div>

<div class="pull-right">
<h3>(3 Punkte)</h3>
</div>

Gegeben ist eine Formel im Domänenkalkül: <br>

$\{ n \mid \exists pid(Playlist(pid, n)$ <br>
$\land \exists tid_1, tid_2(PlaylistTrack(pid, tid_1) \land PlaylistTrack(pid, tid_2)$ <br>
$\land Track(tid_1, 'Kirari', \_, \_, \_, \_, \_, \_)$ <br>
$\land Track(tid_2, 'Mondo Bongo', \_, \_, \_, \_, \_, \_) \land \neg (tid_1 = tid_2))) \}$ <br>


Welche der folgenden Formeln im Tupelkalkül entspricht/entsprechen der obigen Formel?<br>

__a:__ <br>
$\{[p.\textit{Name}] \mid p \in \textit{Playlist}$ <br>
$\land \exists pt (pt.\textit{PlaylistId} = p.\textit{PlaylistId}$ <br>
$\land \exists t_1 \in \textit{Track}(t_1.\textit{TrackId} = p.\textit{TrackId} \land t_1.\textit{Name} = 'Kirari'$ <br>
$\land \exists t_2 \in \textit{Track}(t_2.\textit{TrackId} = p.\textit{TrackId} \land t_2.\textit{Name} = 'Mondo Bongo'$ <br>
$\land \neg (t_1.\textit{TrackId} = t_2.\textit{TrackId}))))\}$<br>


__b:__ <br>
$\{[p.\textit{Name}] \mid p \in \textit{Playlist}$ <br>
$\land \exists pt_1 \in \textit{PlaylistTrack}(pt_1.\textit{PlaylistId} = p.\textit{PlaylistId} $<br>
$\land \exists pt_2 \in \textit{PlaylistTrack}(pt_2.\textit{PlaylistId} = p.\textit{PlaylistId} $<br>
$\land \exists t_1 \in \textit{Track}(t_1.\textit{TrackId} = pt_1.\textit{TrackId} \land t_1.\textit{Name} = 'Kirari' $<br>
$\land \exists t_2 \in \textit{Track}(t_2.\textit{TrackId} = pt_2.\textit{TrackId} \land t_2.\textit{Name} = 'Mondo Bongo' $<br>
$\land \neg (t_1.\textit{TrackId} = t_2.\textit{TrackId})))))\}$


__c:__ <br>
$\{[p.\textit{Name}] \mid p \in \textit{Playlist}$ <br>
$\land \exists pt_1 \in \textit{PlaylistTrack}(pt_1.\textit{PlaylistId} = p.\textit{PlaylistId}$ <br>
$\land \exists t_1 \in \textit{Track}(t_1.\textit{TrackId} = pt_1.\textit{TrackId}$ <br>
$\land (t_1.\textit{Name} = 'Mondo Bongo' \lor t_1.\textit{Name} = 'Kirari')))\}$


__d:__ <br>
$\{[p.\textit{Name}] \mid p \in \textit{PlaylistTrack}$ <br>
$\land \exists t_1 \in \textit{Track}(t_1.\textit{TrackId} = p.\textit{TrackId} \land t_1.\textit{Name} = 'Kirari'$<br>
$\land \exists t_2 \in \textit{Track}(t_2.\textit{TrackId} = p.\textit{TrackId} \land t_2.\textit{Name} = 'Mondo Bongo'$<br>
$\land \neg (t_1.\textit{TrackId} = t_2.\textit{TrackId})))))\}$

In [14]:
 # result ist hier eine String-Menge, z.B. result = {'a','b'}. Füllen sie die Menge unter dem Kommentar mit Ihrer Lösung.
result = set()

# IHR CODE HIER
#b ist das richtige
result = {"b"}
#result = {}
          
          
# nicht : d, a

### DIESEN TEIL NICHT VERÄNDERN, SONST GIBT ES 0 PUNKTE!
subtask = SubTask( task3, "b", points = 3 )

tasks_ready[subtask.task.task][subtask.subtask] = True
for element in result:
    if not isinstance(element, str):
        tasks_ready[subtask.task.task][subtask.subtask] = False
        warnings.warn(f"Die Menge darf nur aus Strings bestehen.")
        break
    if not element in ["a","b","c","d"]:
        tasks_ready[subtask.task.task][subtask.subtask] = False
        warnings.warn(f"Die Menge darf nur die Antwortmöglichkeiten a,b,c,d enthalten.")
        break

subtask.setSolution(result)
subtask.setData(result)

In [ ]:
 # Autograder, bitte ignorieren

# Zusammenfassung

Die folgende Codezelle erlaubt es, die Teilaufgaben 4.1, 4.2 und 4.3 auf syntaktische Korrektheit (z.B. falsche Assertions oder Tippfehler) zu prüfen. Bitte benutzen Sie diese Codezelle, um Ihre Abgaben zu überprüfen.Die Punktzahlangaben in dieser Codezelle dienen nur der Orientierung und geben die __maximal erreichbaren__ Punkte an, nicht die tatsächlich erreichten. 

In [15]:
tasks = ["4.1", "4.2", "4.3"]
points = 0
for t in tasks:
    display(HTML(f"<div class='alert alert-info'><h4>Sammle Ergebnisse von Teilaufgabe {t}:</h4></div>"))

    task = exc.getTaskByLabel(t)
    subtasks = task.getSubtasks()
    for i in subtasks:
        display(HTML(f"<div class='alert alert-info'><h4>Sammle Ergebnisse von Teilaufgabe {t}{subtasks[i].subtask}):</h4></div>"))
        try:
            subTaskGraph = subtasks[i].getData()
        except Exception as e:
            display(HTML(f"<div class='alert alert-danger'> Teilaufgabe {t}{subtasks[i].subtask}) wurde nicht ordnungsgemäß ausgeführt.<br /> Überprüfen Sie die Warnings. </div>"))
            continue
        else:
            if not tasks_ready[subtasks[i].task.task][subtasks[i].subtask]:
                display(HTML(f"<div class='alert alert-danger'> Teilaufgabe {t}{subtasks[i].subtask}) wurde nicht ordnungsgemäß ausgeführt.<br /> Überprüfen Sie die Warnings. </div>"))
                continue
            points += subtasks[i].getPoints()
            display(HTML(f"<div class='alert alert-success'> ... BEARBEITET (+max. {subtasks[i].getPoints()} Punkte) </div>"))

display(HTML(f"<div class='alert alert-success'><h3>Es wurden insgesamt Teilaufgaben für max. {points}/20 Punkte gefunden.</h3></div><hr />"))